# Semantic Segmentation


In [ ]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

## Check for a GPU

In [ ]:
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

## Check TensorFlow Version


In [ ]:
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

In [ ]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    # Refer https://stackoverflow.com/questions/45705070/how-to-load-and-use-a-saved-model-on-tensorflow
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob   = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out  = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out  = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out  = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

In [ ]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """

    std_dev = 0.001
    reg = 0.0001
    
    # 1x1 Convolutions

    conx_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer3")

    conx_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer4")

    conx_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer7")      

    upsample_2x_l7 = tf.layers.conv2d_transpose(vgg_layer7_out, num_classes, 4, strides = (2, 2),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_l7")
    
    fuse1 = tf.add(upsample_2x_l7, conx_1x1_layer4)

    upsample_2x_f1 = tf.layers.conv2d_transpose(fuse1, num_classes, 4, strides = (2, 2),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f1")
    
    fuse2 = tf.add(upsample_2x_f1, conx_1x1_layer3)
    
    upsample_2x_f2 = tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides = (8, 8),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f2")
    
    return upsample_2x_f2

In [ ]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))

    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss_operation = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_operation = optimizer.minimize(loss_operation)
    
    return logits, training_operation, loss_operation

In [ ]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    for i in range(epochs):
        for images, labels in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict={input_image : images, 
                                          correct_label : labels,
                                          keep_prob: 0.5,
                                          learning_rate : 0.0001})
            print('Epoch {}/{}; Training Loss:{:.03f}'.format(i+1, epochs, loss))

In [ ]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        correct_label = tf.placeholder(dtype=tf.float32, shape=(None, None, None, num_classes), name='correct_label')
        learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)

        outputs = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        logits, training_operation, loss_operation = optimize(outputs, correct_label, learning_rate, num_classes)

        epochs = 20
        batch_size = 10
        # TODO: Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        
        train_nn(sess, epochs, batch_size, get_batches_fn, training_operation, loss_operation, input_image, correct_label, keep_prob, learning_rate)

        saver = tf.train.Saver()
        saver.save(sess, './fcn_ss')
        print("Model saved")

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        # OPTIONAL: Apply the trained model to a video

In [ ]:
run()